In [17]:
# Example 6-1. Implementation Order class with pluggable discount strategies
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    def total(self):
        return self.price * self.quantity


class Order:  # the Context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

class Promotion(ABC):  # the Strategy: an Abstract Base Class
    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""

class FidelityPromo(Promotion):  # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):  # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount


class LargeOrderPromo(Promotion):  # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5), LineItem('apple', 10, 1.5), LineItem('watermellon', 5, 5.0)]
print("Order(joe, cart, FidelityPromo()) : ", Order(joe, cart, FidelityPromo()))
print("Order(ann, cart, FidelityPromo()) : ", Order(ann, cart, FidelityPromo()))

banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]
print("Order(joe, banana_cart, BulkItemPromo()) : ", Order(joe, banana_cart, BulkItemPromo()))

long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
print("Order(joe, long_order, LargeOrderPromo()) : ", Order(joe, long_order, LargeOrderPromo()))

Order(joe, cart, FidelityPromo()) :  <Order total: 42.00 due: 42.00>
Order(ann, cart, FidelityPromo()) :  <Order total: 42.00 due: 39.90>
Order(joe, banana_cart, BulkItemPromo()) :  <Order total: 30.00 due: 28.50>
Order(joe, long_order, LargeOrderPromo()) :  <Order total: 10.00 due: 9.30>


In [19]:
# Example 6-3. Order class with discount strategies implemented as functions
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    def total(self):
        return self.price * self.quantity

class Order:  # the Context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)  # <1>
        return self.total() - discount
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

  # <2>
    
def fidelity_promo(order):  # <3>
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5), LineItem('apple', 10, 1.5), LineItem('watermellon', 5, 5.0)]
print("Order(joe, cart, fidelity_promo : ", Order(joe, cart, fidelity_promo))
print("Order(ann, cart, fidelity_promo) : ", Order(ann, cart, fidelity_promo))

banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]
print("Order(joe, banana_cart, bulk_item_promo) : ", Order(joe, banana_cart, bulk_item_promo))

long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
print("Order(joe, long_order, large_order_promo) : ", Order(joe, long_order, large_order_promo))

Order(joe, cart, fidelity_promo :  <Order total: 42.00 due: 42.00>
Order(ann, cart, fidelity_promo) :  <Order total: 42.00 due: 39.90>
Order(joe, banana_cart, bulk_item_promo) :  <Order total: 30.00 due: 28.50>
Order(joe, long_order, large_order_promo) :  <Order total: 10.00 due: 9.30>


In [23]:
# Example 6-5. The best_promo function applies all discounts and returns the largest.
promos = [fidelity_promo, bulk_item_promo, large_order_promo]
def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)
print("Order(joe, long_order, best_promo) : ", Order(joe, long_order, best_promo))
print("Order(joe, banana_cart, best_promo) : ", Order(joe, banana_cart, best_promo))
print("Order(ann, cart, best_promo) : ", Order(ann, cart, best_promo))

Order(joe, long_order, best_promo) :  <Order total: 10.00 due: 9.30>
Order(joe, banana_cart, best_promo) :  <Order total: 30.00 due: 28.50>
Order(ann, cart, best_promo) :  <Order total: 42.00 due: 39.90>


In [28]:
# Example 6-7. The promos list is built by introspection of the module global namespace
promos = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']
print("promos : ", promos)
def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)
print("Order(joe, long_order, best_promo) : ", Order(joe, long_order, best_promo))
print("Order(joe, banana_cart, best_promo) : ", Order(joe, banana_cart, best_promo))
print("Order(ann, cart, best_promo) : ", Order(ann, cart, best_promo))

promos :  [<function fidelity_promo at 0x7f132c566d90>, <function large_order_promo at 0x7f132c5661e0>, <function bulk_item_promo at 0x7f132c566158>]
Order(joe, long_order, best_promo) :  <Order total: 10.00 due: 9.30>
Order(joe, banana_cart, best_promo) :  <Order total: 30.00 due: 28.50>
Order(ann, cart, best_promo) :  <Order total: 42.00 due: 39.90>


In [38]:
# Example 6-8. The promos list is built by introspection of a new promotions module
import promotions
import inspect
promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]
print(promos)

[<function bulk_item_promo at 0x7f132c436ea0>, <function fidelity_promo at 0x7f132c436e18>, <function large_order_promo at 0x7f132c436f28>]


In [ ]:
# Example 6-9. Each instance of MacroCommand has an internal list of commands.
class MacroCommand:
    """A command that executes a list of commands"""
    def __init__(self, commands):
        self.commands = list(commands) # 1
    def __call__(self):
        for command in self.commands: # 2
            command()